In [27]:
from pickle import NONE
from re import T
import pandas as pd
import json
import glob
import os 
import argparse
import logging
import numpy as np


import datetime
import gc

In [28]:
dataset = 'ChnSentiCorp'
model_id = 'bert'
model_folder = model_id + '_trained_models/' + dataset + '/'
faithful_folder = model_id + '_faithfulness/' + dataset + '/'

In [29]:
file_path = f'{faithful_folder}/topk-faithfulness-scores-average-description.json'
topk = pd.read_json(file_path, orient ='index')
topk.rename(columns = {'AOPC - sufficiency':'AOPC_sufficiency', 'AOPC - comprehensiveness':'AOPC_comprehensiveness'}, inplace = True)


In [30]:
topk

,sufficiencies @ 0.01,comprehensiveness @ 0.01,sufficiencies @ 0.02,comprehensiveness @ 0.02,sufficiencies @ 0.05,comprehensiveness @ 0.05,sufficiencies @ 0.1,comprehensiveness @ 0.1,sufficiencies @ 0.2,comprehensiveness @ 0.2,sufficiencies @ 0.5,comprehensiveness @ 0.5,sufficiencies @ 1.0,comprehensiveness @ 1.0,AOPC_sufficiency,AOPC_comprehensiveness
ig,"{'mean': 0.30137762644286803, 'std': 0.3532858...","{'mean': 0.039393287066879, 'std': 0.176197030...","{'mean': 0.293143636391051, 'std': 0.350973745...","{'mean': 0.05024740432671201, 'std': 0.1936873...","{'mean': 0.278383299810607, 'std': 0.323156922...","{'mean': 0.08636525149978701, 'std': 0.2509735...","{'mean': 0.296983793301972, 'std': 0.302800488...","{'mean': 0.121015124769869, 'std': 0.290311926...","{'mean': 0.31758514091170803, 'std': 0.3184279...","{'mean': 0.19389218225889102, 'std': 0.3513425...","{'mean': 0.763494020069288, 'std': 0.347948933...","{'mean': 0.454802356503942, 'std': 0.414878553...","{'mean': 0.9999978868418621, 'std': 2.75692091...","{'mean': 0.6888415874933651, 'std': 0.35686822...","{'mean': 0.46442362910990803, 'std': 0.2365776...","{'mean': 0.233508170559921, 'std': 0.191229409..."
gradients,"{'mean': 0.297896484179559, 'std': 0.353181861...","{'mean': 0.031932753001454005, 'std': 0.156810...","{'mean': 0.279391181149868, 'std': 0.343161134...","{'mean': 0.036577415719355, 'std': 0.164014501...","{'mean': 0.24820162844569701, 'std': 0.3082976...","{'mean': 0.048069685636004, 'std': 0.186040125...","{'mean': 0.243028949176387, 'std': 0.281981562...","{'mean': 0.06461465113958001, 'std': 0.2135127...","{'mean': 0.251411572352717, 'std': 0.291013253...","{'mean': 0.122013876704505, 'std': 0.282918931...","{'mean': 0.705403312116018, 'std': 0.366168886...","{'mean': 0.348073704937908, 'std': 0.390922339...","{'mean': 0.999998554516013, 'std': 1.841244802...","{'mean': 0.688664982972027, 'std': 0.356977008...","{'mean': 0.432190240276608, 'std': 0.224882635...","{'mean': 0.191421010015833, 'std': 0.157102658..."
attention,"{'mean': 0.29581532872993904, 'std': 0.3539588...","{'mean': 0.021156957885785, 'std': 0.129401506...","{'mean': 0.293598284942776, 'std': 0.353798643...","{'mean': 0.034013407793090006, 'std': 0.159172...","{'mean': 0.29219011599092004, 'std': 0.3483728...","{'mean': 0.064587127843534, 'std': 0.212792402...","{'mean': 0.317227146420238, 'std': 0.348149536...","{'mean': 0.10999862462682601, 'std': 0.2709749...","{'mean': 0.44350138760824204, 'std': 0.3693651...","{'mean': 0.176327369394949, 'std': 0.331095069...","{'mean': 0.846323305338694, 'std': 0.266994726...","{'mean': 0.47150888543540204, 'std': 0.4092551...","{'mean': 0.9999988902901441, 'std': 1.38374851...","{'mean': 0.688488763988774, 'std': 0.357085604...","{'mean': 0.498379208474422, 'std': 0.231821657...","{'mean': 0.22372587670976601, 'std': 0.1698592..."
deeplift,"{'mean': 0.297674944374397, 'std': 0.343454378...","{'mean': 0.014415753507109, 'std': 0.097102500...","{'mean': 0.271640708896389, 'std': 0.319828667...","{'mean': 0.021334993398154003, 'std': 0.119749...","{'mean': 0.22114627695770803, 'std': 0.2780301...","{'mean': 0.037009266565139004, 'std': 0.151628...","{'mean': 0.21064964961492402, 'std': 0.2468761...","{'mean': 0.047552330330507, 'std': 0.169808229...","{'mean': 0.232362396991061, 'std': 0.252179564...","{'mean': 0.081006340544976, 'std': 0.224852083...","{'mean': 0.7025399086236761, 'std': 0.37288284...","{'mean': 0.384750570955249, 'std': 0.409488469...","{'mean': 0.999999093007728, 'std': 1.109388723...","{'mean': 0.6883129290954221, 'std': 0.35719401...","{'mean': 0.41943042549512605, 'std': 0.2024381...","{'mean': 0.18205459777093702, 'std': 0.1380559..."
scaled attention,"{'mean': 0.29615082447331803, 'std': 0.3540119...","{'mean': 0.021029671011077, 'std': 0.129057773...","{'mean': 0.294115510226343, 'std': 0.354382548...","{'mean': 0.033137578527329, 'std': 0.157788696...","{'mean': 0.291562421939953, 'std': 0.347151604...","{'mean'

In [31]:
suff = []
comp = []
suff_std = []
comp_std = []
fea_list = ['random', 'attention', "scaled attention", "gradients", "ig", "deeplift"] #"gradientshap", 
for feat in fea_list:
        suff.append(topk.AOPC_sufficiency[str(feat)].get('mean'))
        comp.append(topk.AOPC_comprehensiveness[str(feat)].get('mean'))
        
        suff_std.append(topk.AOPC_sufficiency[str(feat)].get('std'))
        comp_std.append(topk.AOPC_comprehensiveness[str(feat)].get('std'))


In [32]:

fea_list = ['Random', 'Attention', "Scaled Attention", "Gradients", "Integrated Gradients", "Deeplift"]
final_df = pd.DataFrame(list(zip(fea_list, suff, suff_std, comp, comp_std)),
        columns =['Feature','AOPC NormSuff', 'std', 'AOPC NormComp', 'std'])
final_df['Dataset'] = str(dataset)

In [33]:
df = final_df.T
df = df.rename(columns=df.iloc[0]).drop(df.index[0])


In [35]:
df

,Random,Attention,Scaled Attention,Gradients,Integrated Gradients,Deeplift
AOPC NormSuff,0.414919,0.498379,0.498821,0.43219,0.464424,0.41943
std,0.221109,0.231822,0.232416,0.224883,0.236578,0.202438
AOPC NormComp,0.17367,0.223726,0.223349,0.191421,0.233508,0.182055
std,0.136219,0.169859,0.169721,0.157103,0.191229,0.138056
Dataset,ChnSentiCorp,ChnSentiCorp,ChnSentiCorp,ChnSentiCorp,ChnSentiCorp,ChnSentiCorp


In [36]:
df['Attention'] = df['Attention']/df['Random']
# df['Scaled Attention'] = df['Scaled Attention']/df['Random']
# df['Gradients'] = df['Gradients']/df['Random']
# df['Integrated Gradients'] = df['Integrated Gradients']/df['Random']
# df['Deeplift'] = df['Deeplift']/df['Random']

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [34]:
def generate_csv(dataset, method, normal, std):
    file_path = f'./posthoc_results/{dataset}/topk-faithfulness-scores-average-description.json'
    topk = pd.read_json(file_path, orient ='index')
    topk.rename(columns = {'AOPC - sufficiency':'AOPC_sufficiency', 'AOPC - comprehensiveness':'AOPC_comprehensiveness'}, inplace = True)

    file_path = f'./posthoc_results/{dataset}/{method}-faithfulness-scores-normal_{normal}.json'
    soft = pd.read_json(file_path, orient ='index')
    soft.rename(columns = {'sufficiencies @ 1.0':'SoftSuff', 'comprehensiveness @ 1.0':'SoftComp'}, inplace = True)
    
    suff = []
    comp = []
    soft_suff = []
    soft_comp = []
    suff_std = []
    comp_std = []
    soft_suff_std = []
    soft_comp_std = []


    fea_list = ['random', 'attention', "scaled attention", "gradients", "ig", "deeplift"] #"gradientshap", 
    for feat in fea_list:
        suff.append(topk.AOPC_sufficiency[str(feat)].get('mean'))
        comp.append(topk.AOPC_comprehensiveness[str(feat)].get('mean'))
        soft_suff.append(soft.SoftSuff[str(feat)].get('mean'))
        soft_comp.append(soft.SoftComp[str(feat)].get('mean'))

        suff_std.append(topk.AOPC_sufficiency[str(feat)].get('std'))
        comp_std.append(topk.AOPC_comprehensiveness[str(feat)].get('std'))
        soft_suff_std.append(soft.SoftSuff[str(feat)].get('std'))
        soft_comp_std.append(soft.SoftComp[str(feat)].get('std'))


    # if method != 'topk':
    #     random_suff = df.sufficiency['random'].get('mean')
    #     random_comp = df.comprehensiveness['random'].get('mean')
        
    #     Suff_ratio = [x / random_suff for x in sufficiency_mean]
    #     Comp_ratio = [x / random_comp for x in comprehensiveness_mean]

    #     final_df = pd.DataFrame(list(zip(fea_list, sufficiency_mean, Suff_ratio, comprehensiveness_mean, Comp_ratio)),
    #             columns =['Feature', 'Soft-NormSuff', 'Suff_ratio', 'Soft_comprehensiveness', 'Comp_ratio'])
        
    
    #     if 'NOISE' in method: final_path = faithful_result + dataset + '/' + str(method) + str(std) +'_faithfulness_result.csv'
    #     else: final_path = faithful_result + dataset + '/' + str(method) +'_faithfulness_result.csv'
        
    #     final_df.to_csv(final_path)
    #     print('saved csv: ', final_path)

    # else: # not soft, so have aopc
    #     random_suff = df.AOPC_sufficiency['random'].get('mean')
    #     random_comp = df.AOPC_comprehensiveness['random'].get('mean')

    #     Suff_ratio = [x / random_suff for x in sufficiency_mean]
    #     Comp_ratio = [x / random_comp for x in comprehensiveness_mean]
    fea_list = ['Random', 'Attention', "Scaled Attention", "Gradients", "Integrated Gradients", "Deeplift"]

    final_df = pd.DataFrame(list(zip(fea_list, soft_suff, soft_suff_std, soft_comp, soft_comp_std, 
                                        suff, suff_std, comp, comp_std)),
            columns =['Feature', 'Soft-NormSuff', 'std', 'Soft-NormComp', 'std', 
                                ' AOPC NormSuff', 'std', 'AOPC NormComp', 'std'])
    final_df['Dataset'] = str(dataset)


    return final_df


df1 = generate_csv(dataset='sst', method='ZEROOUT', normal=1, std=0.5)
df2 = generate_csv(dataset='agnews', method='ZEROOUT', normal=1, std=0.5)
#df3 = generate_csv(dataset='multirc', method='ZEROOUT', normal=1, std=0.5)
#df4 = generate_csv(dataset='evinf', method='ZEROOUT', normal=1, std=0.5)

final_df = pd.concat([df1, df2])
final_path = './posthoc_results/faithfulness_result.csv'
print('saved csv: ', final_path)
final_df.to_csv(final_path)

FileNotFoundError: File ./posthoc_results/sst/topk-faithfulness-scores-average-description.json does not exist